In [2]:
gene=Array.new #Creating an empty array for saving each line of the document
File.foreach("intensive_integration/documents/ArabidopsisSubNetwork_GeneList.txt"){ |line|  gene <<line } #Saving each line of the document
                                                      # in the array. Source: https://www.rubyguides.com/2015/05/working-with-files-ruby/
gene_p=Array.new   #Creating an empty array for saving procesed name of genes (it has metacharacter \n)
gene.each do |line| 
  gene_p << line.gsub("\n",'') #Eliminating metacharacter \n in each line. 
end   #The difference between gene and gene_p array is that in the second one we can access to each value gene_p[1]
     # and the return has not metacharacter \n

["AT4g27030\n", "AT5g54270\n", "AT1g21400\n", "AT5g19120\n", "AT2g13360\n", "AT4g05180\n", "AT1g22690\n", "AT2g45170\n", "AT4g09650\n", "AT5g55620\n", "AT1g31330\n", "AT1g80440\n", "AT3g28270\n", "AT4g12800\n", "AT5g04140\n", "AT5g64040\n", "AT1g29660\n", "AT1g15820\n", "AT1g64500\n", "AT1g03130\n", "AT2g20670\n", "AT4g17090\n", "AT1g23740\n", "AT2g46340\n", "AT4g17460\n", "AT1g07010\n", "AT1g32220\n", "AT2g03750\n", "AT3g47470\n", "AT4g12830\n", "AT5g05690\n", "AT5g64410\n", "AT2g21330\n", "AT1g15980\n", "AT2g01890\n", "AT1g12250\n", "AT2g21210\n", "AT4g28660\n", "AT1g52230\n", "AT3g18890\n", "AT4g22890\n", "AT1g11850\n", "AT1g37130\n", "AT2g04039\n", "AT3g48420\n", "AT4g19170\n", "AT5g07020\n", "AT5g65010\n", "AT3g01500\n", "AT1g25230\n", "AT2g34430\n", "AT1g18060\n", "AT2g30600\n", "AT4g33010\n", "AT1g65490\n", "AT3g21670\n", "AT4g27260\n", "AT1g12780\n", "AT1g55480\n", "AT2g17880\n", "AT3g49160\n", "AT4g21210\n", "AT5g14740\n", "AT4g37980\n", "AT1g29395\n", "AT2g39730\n", "AT1g2002

In [3]:
print gene_p[..10]
puts
print gene[..10]

["AT4g27030", "AT5g54270", "AT1g21400", "AT5g19120", "AT2g13360", "AT4g05180", "AT1g22690", "AT2g45170", "AT4g09650", "AT5g55620", "AT1g31330"]
["AT4g27030\n", "AT5g54270\n", "AT1g21400\n", "AT5g19120\n", "AT2g13360\n", "AT4g05180\n", "AT1g22690\n", "AT2g45170\n", "AT4g09650\n", "AT5g55620\n", "AT1g31330\n"]

Haria falta un bucle para extraer el uniprot kb de cada gen:

El siguiente bucle devuelve todas las intereacciones que tienen todas las proteinas de los genes de arriba (hria falta limpiar y quitar uniprotkb: y quedarnos con el numero)

Con el  objetivo de buscar las interacciones que tiene cada uno de los genes por separado.

Mas adelante se pueden unir estos dos bucles en uno y hacerlo todo de un tiron

#### He creado este script para trastear y aprenderme el pipeline

In [50]:
gene_p.each do |gene_id|
  address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
  response = RestClient::Request.execute(method: :get, url: address)
  intact_data = response.body.split(/\n/)
  num_inter = intact_data.length    # Number of interactions
  
  print "\n\n", gene_id.to_s, " records: ", num_inter.to_s

  record_list = Array.new

  intact_data.each do |record|
    intact_gene1 = record.split(/\t/)[2].split(/\|/).grep(/^ensemblplants/)  # We filter to extract ensembleplants id
    intact_gene2 = record.split(/\t/)[3].split(/\|/).grep(/^ensemblplants/)  # from columns 2 and 3.
    intact_genes = intact_gene1 + intact_gene2                               # Combine records from both columns. 
    
    score = record.split(/\t/)[-1].split(/\:/)[-1]   # Extract the interaction score of each record.
    
    # For some genes, we obtain interaction reports for different splicing variants (.1, .2, .3)
    # For each gene record, we extract only the gene ID using regular expressions
    
    intact_genes_filtered = Array.new
    intact_genes.each {|splicing| intact_genes_filtered << splicing.match(/AT\dG\d*/).to_s}
    
    # Finally, we remove duplicates from our final interaction reports.
    
    intact_genes_filtered.uniq!
    
    #print "\n\n> Record:\n", intact_genes_filtered
    #print "\nScore: ", score
    
''' With this process, we obtain a set of inteaction records for each gene, each record containing one
    or more interactors. Different records might share common interactors and even contain the query gene.
    
    We want to obtain a final list of unique interactors for each query gene, so we need to remove interactor
    redundancy and interaction of the query gene with itself.
'''
    # Introduce a quality filter !!!!!!
    
    record_list << intact_genes_filtered
  
  end  
  
  print "\n", record_list
  
end                                      



AT4g27030 interactions: 0
[]

AT5g54270 interactions: 0
[]

AT1g21400 interactions: 0
[]

AT5g19120 interactions: 1
[["AT3G46090"]]

AT2g13360 interactions: 4
[["AT2G13360"], ["AT2G13360", "AT3G14415"], ["AT3G46090", "AT2G13360"], ["AT2G01950", "AT2G13360"]]

AT4g05180 interactions: 1
[["AT4G25200", "AT4G05180"]]

AT1g22690 interactions: 0
[]

AT2g45170 interactions: 0
[]

AT4g09650 interactions: 6
[["AT4G37260", "AT4G09650"], ["AT4G32570", "AT4G09650"], ["AT4G37260", "AT4G09650"], ["AT4G32570", "AT4G09650"], ["AT4G37260", "AT4G09650"], ["AT4G32570", "AT4G09650"]]

AT5g55620 interactions: 0
[]

AT1g31330 interactions: 0
[]

AT1g80440 interactions: 0
[]

AT3g28270 interactions: 0
[]

AT4g12800 interactions: 1
[["AT4G12800", "AT2G46820"]]

AT5g04140 interactions: 3
[["AT5G04140", "AT4G37930"], ["AT4G37930", "AT5G04140"], ["AT1G78300", "AT5G04140"]]

AT5g64040 interactions: 1
[["AT5G64040"]]

AT1g29660 interactions: 0
[]

AT1g15820 interactions: 0
[]

AT1g64500 interactions: 0
[]

AT1g0

[["AT5G15850"], ["AT5G15850"], ["AT5G15850", "AT2G46340"], ["AT2G46340", "AT5G15850"], ["AT3G15354", "AT5G15850"], ["AT1G53090", "AT5G15850"], ["AT5G15850", "AT3G15354"], ["AT5G15850", "AT1G53090"], ["AT2G46340", "AT5G15850"], ["AT2G46340", "AT5G15850"], ["AT3G15354", "AT5G15850"], ["AT1G53090", "AT5G15850"], ["AT2G46340", "AT5G15850"], ["AT3G15354", "AT5G15850"], ["AT1G53090", "AT5G15850"], ["AT2G32950", "AT5G15850"], ["AT2G32950", "AT5G15850"], ["AT2G32950", "AT5G15850"], ["AT5G15850", "AT2G32950"], ["AT5G15850", "AT2G46340"], ["AT5G15850", "AT3G15354"], ["AT5G15850", "AT1G53090"], ["AT5G15850", "AT2G32950"], ["AT5G15850", "AT5G14250"], ["AT5G15850", "AT2G46340"], ["AT5G15850", "AT3G15354"], ["AT5G15850", "AT1G53090"], ["AT5G15850", "AT4G10240"], ["AT5G27910", "AT5G15850"], ["AT5G15850", "AT1G08970"], ["AT5G15850", "AT2G46340"], ["AT5G15850", "AT4G11110"], ["AT5G15850", "AT3G15354"], ["AT1G53090", "AT5G15850"], ["AT5G15850", "AT2G46340"], ["AT2G46340", "AT5G15850"], ["AT5G15850", "AT

[["AT3G28220", "AT3G26200"], ["AT3G28220", "AT2G35520"], ["AT3G28220", "AT4G04860"], ["AT3G28220", "AT4G31840"], ["AT3G28220", "AT5G07475"], ["AT3G28220", "AT1G36050"], ["AT3G28220", "AT2G24170"], ["AT3G28220", "AT2G01970"], ["AT3G28220", "AT1G10030"], ["AT3G28220", "AT5G66830"], ["AT3G28220", "AT4G12950"], ["AT3G28220", "AT1G30080"], ["AT3G28220", "AT4G28330"], ["AT3G28220", "AT4G14420"], ["AT3G28220", "AT1G54320"], ["AT3G28220", "AT4G14480"], ["AT3G28220", "AT1G33100"], ["AT3G28220", "AT5G52050"], ["AT3G28220", "AT5G49130"], ["AT3G28220", "AT2G23093"], ["AT3G28220", "AT2G03620"], ["AT3G28220", "AT3G53210"], ["AT3G28220", "AT4G16620"], ["AT3G28220", "AT1G01230"], ["AT3G28220", "AT3G53190"], ["AT3G28220", "AT3G15710"], ["AT3G28220", "AT3G18450"], ["AT3G28220", "AT1G62200"], ["AT3G28220", "AT2G02020"], ["AT3G28220", "AT5G24000"], ["AT3G28220", "AT5G27370"], ["AT3G28220", "AT5G27210"], ["AT3G28220", "AT2G29020"], ["AT3G28220", "AT1G57550"], ["AT3G28220", "AT4G30660"], ["AT3G28220", "AT5G

[["AT4G02770"], ["AT4G37260", "AT4G02770"], ["AT4G37260", "AT4G02770"], ["AT1G31880", "AT4G02770"], ["AT4G37260", "AT4G02770"], ["AT1G31880", "AT4G02770"], ["AT2G23290", "AT4G02770"], ["AT3G24140", "AT4G02770"], ["AT4G02770", "AT4G29930"], ["AT1G09660", "AT4G02770"], ["AT4G02770", "AT5G32440"], ["AT3G08530", "AT4G02770"], ["AT4G02770", "AT4G17680"], ["AT4G02770"], ["AT4G19700", "AT4G02770"], ["AT4G02770", "AT4G30080"], ["AT4G02770", "AT3G47620"], ["AT5G37670", "AT4G02770"], ["AT5G24660", "AT4G02770"], ["AT4G02770", "AT5G61010"], ["AT4G02770", "AT1G15670"], ["AT4G02770", "AT1G01260"], ["AT4G02770", "AT3G61150"], ["AT3G49580", "AT4G02770"], ["AT4G02770", "AT3G48150"], ["AT1G31880", "AT4G02770"], ["AT4G22890", "AT4G02770"]]

AT4g39710 interactions: 4
[["AT4G39710", "AT4G13530"], ["AT4G39710", "AT4G03280"], ["AT4G39710", "AT4G03280"], ["AT4G39710", "AT4G03280"]]

AT5g48300 interactions: 2
[["AT5G48300", "AT2G21590"], ["AT3G09250", "AT5G48300"]]

AT5g48490 interactions: 0
[]

AT1g20440 inte

["AT4g27030", "AT5g54270", "AT1g21400", "AT5g19120", "AT2g13360", "AT4g05180", "AT1g22690", "AT2g45170", "AT4g09650", "AT5g55620", "AT1g31330", "AT1g80440", "AT3g28270", "AT4g12800", "AT5g04140", "AT5g64040", "AT1g29660", "AT1g15820", "AT1g64500", "AT1g03130", "AT2g20670", "AT4g17090", "AT1g23740", "AT2g46340", "AT4g17460", "AT1g07010", "AT1g32220", "AT2g03750", "AT3g47470", "AT4g12830", "AT5g05690", "AT5g64410", "AT2g21330", "AT1g15980", "AT2g01890", "AT1g12250", "AT2g21210", "AT4g28660", "AT1g52230", "AT3g18890", "AT4g22890", "AT1g11850", "AT1g37130", "AT2g04039", "AT3g48420", "AT4g19170", "AT5g07020", "AT5g65010", "AT3g01500", "AT1g25230", "AT2g34430", "AT1g18060", "AT2g30600", "AT4g33010", "AT1g65490", "AT3g21670", "AT4g27260", "AT1g12780", "AT1g55480", "AT2g17880", "AT3g49160", "AT4g21210", "AT5g14740", "AT4g37980", "AT1g29395", "AT2g39730", "AT1g20020", "AT2g34420", "AT4g37800", "AT1g70820", "AT3g26740", "AT4g28780", "AT1g12900", "AT1g64680", "AT2g25200", "AT3g55800", "AT4g21280"

In [43]:
require 'rest-client'   # This access to the page of each gene 
  interactor1={}
  interactor2={}
  
  punctuations={}
#We have to search in other data base the interactions
intactstring=Array.new
gene_p.each do |gene_id|
  address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
  response = RestClient::Request.execute(method: :get, url: address)
  record = response.body.split(/\n/)  #split the string to substrings based on new line pattern: https://www.geeksforgeeks.org/ruby-string-split-method-with-examples/
                                      #separating by each interaction
  temp_arr1=[]
  temp_arr2=[]
  punctuation_arr=[]

  record.each do |line|      

    tester1 = line.split(/\t/)[2].split(/\|/).grep(/^ensemblplants/)[0]
    tester2 = line.split(/\t/)[3].split(/\|/).grep(/^ensemblplants/)[0]
        
    punctuation = line.split(/\t/)[-1].split(/\:/)[1]
    
    unless tester1.nil? || tester2.nil? || punctuation.to_f < 0.5 
      temp1 = tester1.match(/:(.*)(.*)/)[1..]
      temp2 = tester2.match(/:(.*)(.*)/)[1..]      
        
      temp_arr1.append(temp1[0])  
      temp_arr2.append(temp2[0])  
      punctuation_arr.append(punctuation)
      
      dup_finder1=temp_arr1.each_index.select { |index| temp_arr1[index] == temp1[0]} 
      dup_finder2=temp_arr2.each_index.select { |index| temp_arr2[index] == temp2[0]} 
      
      ## Remove duplicates
      if (dup_finder1.length > 1 && dup_finder2.length > 1) && \
          ((dup_finder2-dup_finder1).empty? || (dup_finder1-dup_finder2).empty?)
        puts"deleting last interaction: 1: #{temp_arr1} / 2: #{temp_arr2}"
        temp_arr1.pop
        temp_arr2.pop
        punctuation_arr.pop
        puts"DONE 1: #{temp_arr1} , 2: #{temp_arr2}"
      end
    end
    interactor1[gene_id] = temp_arr1
    interactor2[gene_id] = temp_arr2
    punctuations[gene_id] = punctuation_arr
  end

end


deleting last interaction: 1: ["AT4G37260.1", "AT4G32570.1", "AT4G37260.1"] / 2: ["AT4G09650.1", "AT4G09650.1", "AT4G09650.1"]
DONE 1: ["AT4G37260.1", "AT4G32570.1"] , 2: ["AT4G09650.1", "AT4G09650.1"]
deleting last interaction: 1: ["AT4G37260.1", "AT4G32570.1", "AT4G32570.1"] / 2: ["AT4G09650.1", "AT4G09650.1", "AT4G09650.1"]
DONE 1: ["AT4G37260.1", "AT4G32570.1"] , 2: ["AT4G09650.1", "AT4G09650.1"]
deleting last interaction: 1: ["AT4G37260.1", "AT4G32570.1", "AT4G37260.1"] / 2: ["AT4G09650.1", "AT4G09650.1", "AT4G09650.1"]
DONE 1: ["AT4G37260.1", "AT4G32570.1"] , 2: ["AT4G09650.1", "AT4G09650.1"]
deleting last interaction: 1: ["AT4G37260.1", "AT4G32570.1", "AT4G32570.1"] / 2: ["AT4G09650.1", "AT4G09650.1", "AT4G09650.1"]
DONE 1: ["AT4G37260.1", "AT4G32570.1"] , 2: ["AT4G09650.1", "AT4G09650.1"]
deleting last interaction: 1: ["AT4G37260.1", "AT4G37260.1"] / 2: ["AT1G03130.1", "AT1G03130.1"]
DONE 1: ["AT4G37260.1"] , 2: ["AT1G03130.1"]
deleting last interaction: 1: ["AT4G37260.1", "AT4

deleting last interaction: 1: ["AT1G37130.1", "AT5G18930.1", "AT1G31880.1", "AT4G32570.1", "AT1G37130.1", "AT2G37630.1", "AT5G60120.1", "AT4G32570.1"] / 2: ["AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1"]
DONE 1: ["AT1G37130.1", "AT5G18930.1", "AT1G31880.1", "AT4G32570.1", "AT1G37130.1", "AT2G37630.1", "AT5G60120.1"] , 2: ["AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1", "AT1G37130.1", "AT1G37130.1"]
deleting last interaction: 1: ["AT1G37130.1", "AT5G18930.1", "AT1G31880.1", "AT4G32570.1", "AT1G37130.1", "AT2G37630.1", "AT5G60120.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1"] / 2: ["AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1", "AT1G37130.1", "AT1G37130.1", "AT2G37630.1", "AT4G32570.1", "AT1G37130.1"]
DONE 1: ["AT1G37130.1", "AT5G18930.1", "AT1G31880.1", "AT4G32570.1", "AT1G37130.1", "AT2G37630.1", "AT5G60120.1", "AT1G37130.1", "AT1G37130.1"] , 2: ["AT1G37130.

deleting last interaction: 1: ["AT5G15850.1", "AT2G46340.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1", "AT5G15850.1", "AT1G53090.1"] / 2: ["AT2G46340.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1"]
DONE 1: ["AT5G15850.1", "AT2G46340.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1", "AT5G15850.1"] , 2: ["AT2G46340.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1", "AT3G15354.3", "AT1G53090.1"]
deleting last interaction: 1: ["AT5G15850.1", "AT2G46340.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1"] / 2: ["AT2G46340.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1", "AT3G15354.3", "AT1G53090.1", "AT2G46340.1"]
DONE 1: ["AT5G15850.1", "AT2G46340.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1", "AT5G15850.1"] , 2: ["AT2G46340.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1", "AT3G15354.3", "AT1G53090.1"]
deleting last interaction: 1: ["AT5G15850.1", "AT2G46340.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1", "AT5G15850

DONE 1: ["AT3G48360.1", "AT4G09570.1"] , 2: ["AT4G09570.1", "AT3G48360.1"]
deleting last interaction: 1: ["AT3G48360.1", "AT4G09570.1", "AT3G48360.1"] / 2: ["AT4G09570.1", "AT3G48360.1", "AT4G09570.1"]
DONE 1: ["AT3G48360.1", "AT4G09570.1"] , 2: ["AT4G09570.1", "AT3G48360.1"]
deleting last interaction: 1: ["AT3G48360.1", "AT4G09570.1", "AT4G09570.1"] / 2: ["AT4G09570.1", "AT3G48360.1", "AT3G48360.1"]
DONE 1: ["AT3G48360.1", "AT4G09570.1"] , 2: ["AT4G09570.1", "AT3G48360.1"]
deleting last interaction: 1: ["AT1G71030.1", "AT1G71030.1", "AT1G71030.1"] / 2: ["AT4G09820.1", "AT1G60250.1", "AT1G60250.1"]
DONE 1: ["AT1G71030.1", "AT1G71030.1"] , 2: ["AT4G09820.1", "AT1G60250.1"]
deleting last interaction: 1: ["AT1G71030.1", "AT1G71030.1", "AT4G09820.1", "AT1G71030.1"] / 2: ["AT4G09820.1", "AT1G60250.1", "AT1G71030.1", "AT1G60250.1"]
DONE 1: ["AT1G71030.1", "AT1G71030.1", "AT4G09820.1"] , 2: ["AT4G09820.1", "AT1G60250.1", "AT1G71030.1"]
deleting last interaction: 1: ["AT1G71030.1", "AT1G71030.

["AT4g27030", "AT5g54270", "AT1g21400", "AT5g19120", "AT2g13360", "AT4g05180", "AT1g22690", "AT2g45170", "AT4g09650", "AT5g55620", "AT1g31330", "AT1g80440", "AT3g28270", "AT4g12800", "AT5g04140", "AT5g64040", "AT1g29660", "AT1g15820", "AT1g64500", "AT1g03130", "AT2g20670", "AT4g17090", "AT1g23740", "AT2g46340", "AT4g17460", "AT1g07010", "AT1g32220", "AT2g03750", "AT3g47470", "AT4g12830", "AT5g05690", "AT5g64410", "AT2g21330", "AT1g15980", "AT2g01890", "AT1g12250", "AT2g21210", "AT4g28660", "AT1g52230", "AT3g18890", "AT4g22890", "AT1g11850", "AT1g37130", "AT2g04039", "AT3g48420", "AT4g19170", "AT5g07020", "AT5g65010", "AT3g01500", "AT1g25230", "AT2g34430", "AT1g18060", "AT2g30600", "AT4g33010", "AT1g65490", "AT3g21670", "AT4g27260", "AT1g12780", "AT1g55480", "AT2g17880", "AT3g49160", "AT4g21210", "AT5g14740", "AT4g37980", "AT1g29395", "AT2g39730", "AT1g20020", "AT2g34420", "AT4g37800", "AT1g70820", "AT3g26740", "AT4g28780", "AT1g12900", "AT1g64680", "AT2g25200", "AT3g55800", "AT4g21280"

In [44]:
interactor1


{"AT5g19120"=>[], "AT2g13360"=>[], "AT4g05180"=>[], "AT4g09650"=>["AT4G37260.1", "AT4G32570.1"], "AT4g12800"=>[], "AT5g04140"=>["AT5G04140.1", "AT4G37930.1"], "AT5g64040"=>[], "AT1g03130"=>["AT4G37260.1"], "AT4g17460"=>["AT4G17460.1", "AT4G17460.1", "AT4G17460.1", "AT4G17460.1", "AT4G17460.1", "AT2G44910.1"], "AT1g07010"=>[], "AT5g05690"=>["AT5G05690.1"], "AT2g01890"=>[], "AT1g12250"=>[], "AT4g22890"=>[], "AT1g11850"=>[], "AT1g37130"=>["AT1G37130.1", "AT5G18930.1", "AT1G31880.1", "AT4G32570.1", "AT1G37130.1", "AT2G37630.1", "AT5G60120.1", "AT1G37130.1", "AT1G37130.1"], "AT4g19170"=>["AT4G19170.1"], "AT3g01500"=>[], "AT4g33010"=>[], "AT1g12780"=>[], "AT2g17880"=>[], "AT3g49160"=>[], "AT4g21210"=>[], "AT5g14740"=>[], "AT2g39730"=>[], "AT1g20020"=>[], "AT2g34420"=>[], "AT4g28780"=>[], "AT1g12900"=>[], "AT3g55800"=>[], "AT5g15850"=>["AT5G15850.1", "AT2G46340.1", "AT3G15354.3", "AT1G53090.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1"], "AT1g06680"=>[], "AT1g42970"=>[], "AT5g17300"=>["AT5G

In [26]:
interactor2

{"AT5g19120"=>[], "AT2g13360"=>[], "AT4g05180"=>[], "AT4g09650"=>["AT4G09650.1", "AT4G09650.1"], "AT4g12800"=>[], "AT5g04140"=>["AT4G37930.1", "AT5G04140.1"], "AT5g64040"=>[], "AT1g03130"=>["AT1G03130.1"], "AT4g17460"=>["AT1G70920.1", "AT2G44910.1", "AT3G60390.1", "AT3G15030.1", "AT2G22800.1", "AT4G17460.1"], "AT1g07010"=>[], "AT5g05690"=>["AT3G54130.1"], "AT2g01890"=>[], "AT1g12250"=>[], "AT4g22890"=>[], "AT1g11850"=>[], "AT1g37130"=>["AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1", "AT1G37130.1", "AT1G37130.1", "AT2G37630.1", "AT4G32570.1"], "AT4g19170"=>["AT5G17790.1"], "AT3g01500"=>[], "AT4g33010"=>[], "AT1g12780"=>[], "AT2g17880"=>[], "AT3g49160"=>[], "AT4g21210"=>[], "AT5g14740"=>[], "AT2g39730"=>[], "AT1g20020"=>[], "AT2g34420"=>[], "AT4g28780"=>[], "AT1g12900"=>[], "AT3g55800"=>[], "AT5g15850"=>["AT2G46340.1", "AT5G15850.1", "AT5G15850.1", "AT5G15850.1", "AT3G15354.3", "AT1G53090.1", "AT4G10240.1"], "AT1g06680"=>[], "AT1g42970"=>[], "AT5g17300"=>["AT5G

In [311]:
a=["A","A","d","c","F","F","A"]
b=[1,2,3,4,4,5,1]

dup_finder1=[0, 2]#a.each_index.select { |index| a[index] == "A"}
dup_finder2=[0,1,2,3]#b.each_index.select { |index| b[index] == 1}
if (dup_finder1.length > 1 && dup_finder2.length > 1) && \
  ((dup_finder2-dup_finder1).empty? || (dup_finder1-dup_finder2).empty?)
    print dup_finder1,dup_finder2, (dup_finder2-dup_finder1), (dup_finder1-dup_finder2)
#filter = dup_finder1.each_index.filter_map {  |i| i if dup_finder2[i] == dup_finder1[i] }
    a.pop
    b.pop
    p a,b
end


[0, 2][0, 1, 2, 3][1, 3][]["A", "A", "d", "c", "F", "F"]
[1, 2, 3, 4, 4, 5]


[["A", "A", "d", "c", "F", "F"], [1, 2, 3, 4, 4, 5]]

In [265]:
fi = b.each_index.filter_map {  |i| i if a[i] == b[i] }
fi.length

1

In [299]:
interactor2

{"AT5g19120"=>[], "AT2g13360"=>[], "AT4g05180"=>[], "AT4g09650"=>["AT4G09650.1", "AT4G09650.1", "AT4G09650.1", "AT4G09650.1"], "AT4g12800"=>[], "AT5g04140"=>["AT4G37930.1", "AT5G04140.1"], "AT5g64040"=>[], "AT1g03130"=>["AT1G03130.1"], "AT4g17460"=>["AT1G70920.1", "AT1G70920.1", "AT2G44910.1", "AT2G44910.1", "AT3G60390.1", "AT3G60390.1", "AT2G22800.1", "AT3G15030.1", "AT3G15030.1"], "AT1g07010"=>[], "AT5g05690"=>["AT3G54130.1"], "AT2g01890"=>[], "AT1g12250"=>[], "AT4g22890"=>[], "AT1g11850"=>[], "AT1g37130"=>["AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT1G37130.1", "AT5G18930.1", "AT1G37130.1", "AT1G37130.1"], "AT4g19170"=>["AT5G17790.1", "AT5G17790.1"], "AT3g01500"=>[], "AT4g33010"=>[], "AT1g12780"=>[], "AT2g17880"=>[], "AT3g49160"=>[], "AT4g21210"=>[], "AT5g14740"=>[], "AT2g39730"=>[], "AT1g20020"=>[], "AT2g34420"=>[], "AT4g28780"=>[], "AT1g12900"=>[], "AT3g55800"=>[], "AT5g15850"=>["AT2G46340.1", "AT5G1585

In [109]:
interactor1.keys.each do |inter|
  if interactor1[inter].empty?
    puts "No interaction found"
  else
    puts "Interactors 1 with #{inter}"
    puts interactor1[inter]
    puts "2222 Interactors with #{inter}"
    puts interactor2[inter]
    
  end
end

No interaction found
No interaction found
No interaction found
Interactors 1 with AT4g09650
AT4G37260.1
AT4G32570.1
AT4G37260.1
AT4G32570.1
AT4G37260.1
AT4G32570.1
2222 Interactors with AT4g09650
AT4G09650.1
AT4G09650.1
AT4G09650.1
AT4G09650.1
AT4G09650.1
AT4G09650.1
No interaction found
Interactors 1 with AT5g04140
AT5G04140.1
AT4G37930.1
2222 Interactors with AT5g04140
AT4G37930.1
AT5G04140.1
No interaction found
Interactors 1 with AT1g03130
AT4G37260.1
AT4G37260.1
AT4G37260.1
2222 Interactors with AT1g03130
AT1G03130.1
AT1G03130.1
AT1G03130.1
Interactors 1 with AT4g17460
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT4G17460.1
AT2G44910.1
AT4G17460.1
AT4G17460.1
2222 Interactors with AT4g17460
AT1G70920.1
AT1G70920.1
AT2G44910.1
AT2G44910.1
AT3G60390.1
AT3G15030.1
AT3G60390.1
AT2G22800.1
AT1G70920.1
AT3G60390.1
AT2G44910.1
AT2

["AT5g19120", "AT2g13360", "AT4g05180", "AT4g09650", "AT4g12800", "AT5g04140", "AT5g64040", "AT1g03130", "AT4g17460", "AT1g07010", "AT5g05690", "AT2g01890", "AT1g12250", "AT4g22890", "AT1g11850", "AT1g37130", "AT4g19170", "AT3g01500", "AT4g33010", "AT1g12780", "AT2g17880", "AT3g49160", "AT4g21210", "AT5g14740", "AT2g39730", "AT1g20020", "AT2g34420", "AT4g28780", "AT1g12900", "AT3g55800", "AT5g15850", "AT1g06680", "AT1g42970", "AT5g17300", "AT2g26080", "AT5g20250", "AT1g54040", "AT3g21870", "AT3g48360", "AT2g43010", "AT3g28220", "AT5g11070", "AT1g68010", "AT2g26740", "AT1g71030", "AT3g54050", "AT3g45780", "AT2g30950", "AT3g62410", "AT3g16250", "AT1g76080", "AT3g46780", "AT2g33830", "AT5g35630", "AT1g29910", "AT1g69870", "AT2g47930", "AT5g43750", "AT3g50820", "AT1g13930", "AT4g37610", "AT1g76100", "AT3g61260", "AT4g04640", "AT5g38990", "AT1g29920", "AT4g02770", "AT4g39710", "AT5g48300", "AT3g62550", "AT1g14280", "AT4g09010", "AT5g01530"]

In [89]:
puts uniprot_ID

NameError: undefined local variable or method `uniprot_ID' for #<Object:0x000055a990c7ff30 @prompt={:PROMPT_I=>"3.0.0 :%03n > ", :PROMPT_S=>"3.0.0 :%03n%l> ", :PROMPT_C=>"3.0.0 :%03n > ", :PROMPT_N=>"3.0.0 :%03n?> ", :RETURN=>" => %s \n", :AUTO_INDENT=>true}>

Esta linea de abajo es para ir probando con un unico gen

In [183]:
require 'rest-client'
gene_id="At4g18960"
#unless gene_id.match(/AT\dg\ddddd/)
#  abort
#end
address = "http://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/#{gene_id}?format=tab25"
response = RestClient::Request.execute(method: :get, url: address)
record = response.body
record_su= record.split(/\n/) #split the string to substrings based on new line pattern: https://www.geeksforgeeks.org/ruby-string-split-method-with-examples/
                              #separating by each interaction
interactor1=[]
interactor2=[]

punctuations=[]

record_su.each do |line|
  puts "original 14"
  puts line.split(/\t/)[14].split(/\:/)[1]
  puts "new -1"
  puts line.split(/\t/)[-1].split(/\:/)[1]
  #sometimes one interactor is lose, eliminating these
#  unless line[2].split(/\|/).grep(/ensemblplants:.*/) and line[3].split(/\|/).grep(/.*ensemblplants:.*/)
#    next("")
#  else
#    #Aqui quiero que haga el append solo y cuando exista la linea pero no se como ponerlo
#    #Muchos genes están vacíos entonces esa interaccion no se deberia appendar pero no se como meterlo
#    #son las 21.17 de un viernes 25 de noviembre del año 2022. Lo que está dentro del else funciona
#    #si no me creeis miradlo, va fino. El problema es ese, que se appendan lneas vacias.
#    
# #   unless line.split(/\t/)[2].split(/\|/).grep(/^ensemblplants/)[0]? and f line.split(/\t/)[3].split(/\|/).grep(/^ensemblplants/)[0]?
#  #    next
#   # else
#    
#    tester1 = line.split(/\t/)[2].split(/\|/).grep(/^ensemblplants/)[0]
#    tester2 = line.split(/\t/)[3].split(/\|/).grep(/^ensemblplants/)[0]
#    punctuation = line.split(/\t/)[14].split(/\:/)[1]
#    unless tester1.nil? || tester2.nil? || punctuation.to_f < 0.5
#      temp1 = tester1.match(/:(.*)(.*)/)[1..]
#      temp2 = tester2.match(/:(.*)(.*)/)[1..]
#  
#      interactor1.append(temp1[0])
#      interactor2.append(temp2[0])
#      
#      punctuations.append(punctuation)
#    end
#  end
end


#A veces la proteina va primera, a veces va segunda
#A veces la misma interaccion se repite pero con las proteinas cambiadas de orden
#La data base devuelve TODAS las interacciones de esa proteína dada

original 14
0.37
new -1
0.37
original 14
0.67
new -1
0.67
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.67
new -1
0.67
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.67
new -1
0.67
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.37
new -1
0.37
original 14
0.35
new -1
0.35
original 14
0.

["uniprotkb:P17839\tuniprotkb:P17839\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)\tpsi-mi:\"MI:0018\"(two hybrid)\tFavaro et al. (2003)\tpubmed:14555696\ttaxid:3702(arath)|taxid:3702(\"Arabidopsis thaliana (Mouse-ear cress)\")\ttaxid:3702(arath)|taxid:3702(\"Arabidopsis thaliana (Mouse-ear cress)\")\tpsi-mi:\"MI:0915\"(physical association)\tpsi-mi:\"MI:0469\"(IntAct)\tintact:EBI-592383\tintact-miscore:0.37", "uniprotkb:P17839\tuniprotkb:O22456\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tintact:EBI-592020|uniprotkb:Q8GWQ4|uniprotkb:Q8L9R5|ensemblplants:AT1G24260.2\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.

In [78]:
#.match(/___/i) ignora el case sensitive
puts interactor1

AT4G18960.1
AT4G18960.1
AT1G24260.2
AT4G18960.1
AT4G18960.1
AT4G18960.1
AT1G24260.2
AT1G24260.2
AT2G45650.1
AT5G15800.1
AT5G15800.1
AT2G45650.1
AT1G24260.2


In [80]:
puts interactor2

AT1G24260.2
AT1G24260.2
AT4G18960.1
AT1G24260.2
AT5G15800.1
AT2G45650.1
AT4G18960.1
AT4G18960.1
AT4G18960.1
AT4G18960.1
AT4G18960.1
AT4G18960.1
AT4G18960.1


In [81]:
punctuations

["0.67", "0.67", "0.67", "0.67", "0.53", "0.53", "0.53", "0.53", "0.53", "0.53", "0.53", "0.53", "0.53"]

In [199]:
record_su.each do |line|
  puts line.split(/\t/)[14].split(/\:/)[1]
end

0.37
0.67
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.67
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.67
0.37
0.37
0.37
0.37
0.37
0.37
0.37
0.35
0.67
0.37
0.37
0.53
0.37
0.53
0.53
0.37
0.53
0.53
0.53
0.53
0.53
0.53
0.37
0.37
0.49
0.49
0.37
0.37
0.37
0.37
0.37
0.49
0.49


["uniprotkb:P17839\tuniprotkb:P17839\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)\tpsi-mi:\"MI:0018\"(two hybrid)\tFavaro et al. (2003)\tpubmed:14555696\ttaxid:3702(arath)|taxid:3702(\"Arabidopsis thaliana (Mouse-ear cress)\")\ttaxid:3702(arath)|taxid:3702(\"Arabidopsis thaliana (Mouse-ear cress)\")\tpsi-mi:\"MI:0915\"(physical association)\tpsi-mi:\"MI:0469\"(IntAct)\tintact:EBI-592383\tintact-miscore:0.37", "uniprotkb:P17839\tuniprotkb:O22456\tintact:EBI-592365|uniprotkb:O82732|ensemblplants:AT4G18960.1\tintact:EBI-592020|uniprotkb:Q8GWQ4|uniprotkb:Q8L9R5|ensemblplants:AT1G24260.2\tpsi-mi:ag_arath(display_long)|uniprotkb:F13C5.

In [33]:
puts interactor2[0].match(/:(.*)(.*)$/)[1..]

AT4G18960.1



In [21]:
puts record_su[1].match(/:(\S*) (.*)/)

:ag_arath(display_long)|uniprotkb:F13C5.130(orf name)|uniprotkb:AG(gene name)|psi-mi:AG(display_short)|uniprotkb:At4g18960(locus name)	psi-mi:sep3_arath(display_long)|uniprotkb:Agamous-like MADS-box protein AGL9(gene name synonym)|uniprotkb:SEP3(gene name)|psi-mi:SEP3(display_short)|uniprotkb:AGL9(gene name synonym)|uniprotkb:At1g24260(locus name)|uniprotkb:F3I6.19(orf name)	psi-mi:"MI:0018"(two hybrid)	Favaro et al. (2003)	pubmed:14555696	taxid:3702(arath)|taxid:3702("Arabidopsis thaliana (Mouse-ear cress)")	taxid:3702(arath)|taxid:3702("Arabidopsis thaliana (Mouse-ear cress)")	psi-mi:"MI:0915"(physical association)	psi-mi:"MI:0469"(IntAct)	intact:EBI-592374	intact-miscore:0.67
